# Μηχανισμοί προσοχής και μετασχηματιστές

Ένα σημαντικό μειονέκτημα των επαναλαμβανόμενων δικτύων είναι ότι όλες οι λέξεις σε μια ακολουθία έχουν την ίδια επίδραση στο αποτέλεσμα. Αυτό οδηγεί σε υπο-βέλτιστη απόδοση με τα τυπικά μοντέλα LSTM encoder-decoder για εργασίες ακολουθίας προς ακολουθία, όπως η Αναγνώριση Οντοτήτων και η Μετάφραση Κειμένου. Στην πραγματικότητα, συγκεκριμένες λέξεις στην είσοδο συχνά έχουν μεγαλύτερη επίδραση στα αποτελέσματα από άλλες.

Ας εξετάσουμε ένα μοντέλο ακολουθίας προς ακολουθία, όπως η μετάφραση κειμένου. Αυτό υλοποιείται από δύο επαναλαμβανόμενα δίκτυα, όπου το ένα δίκτυο (**encoder**) συμπυκνώνει την είσοδο σε μια κρυφή κατάσταση, και το άλλο (**decoder**) ξεδιπλώνει αυτή την κρυφή κατάσταση στο μεταφρασμένο αποτέλεσμα. Το πρόβλημα με αυτή την προσέγγιση είναι ότι η τελική κατάσταση του δικτύου δυσκολεύεται να θυμηθεί την αρχή της πρότασης, προκαλώντας έτσι χαμηλή ποιότητα του μοντέλου σε μεγάλες προτάσεις.

**Οι Μηχανισμοί Προσοχής** παρέχουν έναν τρόπο να δίνεται βάρος στην επίδραση κάθε εισόδου στο κάθε αποτέλεσμα του RNN. Αυτό υλοποιείται δημιουργώντας συντομεύσεις μεταξύ των ενδιάμεσων καταστάσεων του RNN εισόδου και του RNN εξόδου. Με αυτόν τον τρόπο, κατά τη δημιουργία του συμβόλου εξόδου $y_t$, λαμβάνουμε υπόψη όλες τις κρυφές καταστάσεις εισόδου $h_i$, με διαφορετικούς συντελεστές βάρους $\alpha_{t,i}$.

![Εικόνα που δείχνει ένα μοντέλο encoder/decoder με ένα πρόσθετο επίπεδο προσοχής](../../../../../translated_images/el/encoder-decoder-attention.7a726296894fb567.webp)
*Το μοντέλο encoder-decoder με μηχανισμό πρόσθετης προσοχής στο [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), από [αυτή την ανάρτηση](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Ο πίνακας προσοχής $\{\alpha_{i,j}\}$ αντιπροσωπεύει τον βαθμό στον οποίο συγκεκριμένες λέξεις εισόδου συμβάλλουν στη δημιουργία μιας δεδομένης λέξης στην ακολουθία εξόδου. Παρακάτω είναι ένα παράδειγμα ενός τέτοιου πίνακα:

![Εικόνα που δείχνει ένα δείγμα ευθυγράμμισης από το RNNsearch-50, από Bahdanau - arviz.org](../../../../../translated_images/el/bahdanau-fig3.09ba2d37f202a6af.webp)

*Εικόνα από [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Οι μηχανισμοί προσοχής είναι υπεύθυνοι για μεγάλο μέρος της τρέχουσας ή σχεδόν τρέχουσας κορυφαίας απόδοσης στην Επεξεργασία Φυσικής Γλώσσας. Ωστόσο, η προσθήκη προσοχής αυξάνει σημαντικά τον αριθμό των παραμέτρων του μοντέλου, κάτι που οδήγησε σε προβλήματα κλιμάκωσης με τα RNNs. Ένας βασικός περιορισμός στην κλιμάκωση των RNNs είναι ότι η επαναλαμβανόμενη φύση των μοντέλων καθιστά δύσκολη την ομαδοποίηση και την παράλληλη εκπαίδευση. Σε ένα RNN κάθε στοιχείο μιας ακολουθίας πρέπει να επεξεργαστεί με τη σειρά, κάτι που σημαίνει ότι δεν μπορεί να παραλληλιστεί εύκολα.

Η υιοθέτηση μηχανισμών προσοχής σε συνδυασμό με αυτόν τον περιορισμό οδήγησε στη δημιουργία των κορυφαίων μοντέλων μετασχηματιστών που γνωρίζουμε και χρησιμοποιούμε σήμερα, από το BERT έως το OpenGPT3.

## Μοντέλα μετασχηματιστών

Αντί να προωθούν το πλαίσιο κάθε προηγούμενης πρόβλεψης στο επόμενο βήμα αξιολόγησης, τα **μοντέλα μετασχηματιστών** χρησιμοποιούν **κωδικοποιήσεις θέσης** και **προσοχή** για να καταγράψουν το πλαίσιο μιας δεδομένης εισόδου μέσα σε ένα παρεχόμενο παράθυρο κειμένου. Η παρακάτω εικόνα δείχνει πώς οι κωδικοποιήσεις θέσης με προσοχή μπορούν να καταγράψουν το πλαίσιο μέσα σε ένα δεδομένο παράθυρο.

![Κινούμενο GIF που δείχνει πώς πραγματοποιούνται οι αξιολογήσεις στα μοντέλα μετασχηματιστών.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Επειδή κάθε θέση εισόδου αντιστοιχίζεται ανεξάρτητα σε κάθε θέση εξόδου, οι μετασχηματιστές μπορούν να παραλληλίσουν καλύτερα από τα RNNs, κάτι που επιτρέπει πολύ μεγαλύτερα και πιο εκφραστικά γλωσσικά μοντέλα. Κάθε κεφαλή προσοχής μπορεί να χρησιμοποιηθεί για να μάθει διαφορετικές σχέσεις μεταξύ λέξεων, βελτιώνοντας τις εργασίες Επεξεργασίας Φυσικής Γλώσσας.

## Δημιουργία Απλού Μοντέλου Μετασχηματιστή

Το Keras δεν περιέχει ενσωματωμένο επίπεδο μετασχηματιστή, αλλά μπορούμε να δημιουργήσουμε το δικό μας. Όπως και πριν, θα επικεντρωθούμε στην ταξινόμηση κειμένου του συνόλου δεδομένων AG News, αλλά αξίζει να σημειωθεί ότι τα μοντέλα μετασχηματιστών δείχνουν τα καλύτερα αποτελέσματα σε πιο δύσκολες εργασίες NLP.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Νέα επίπεδα στο Keras πρέπει να υποκατηγοριοποιούν την κλάση `Layer` και να υλοποιούν τη μέθοδο `call`. Ας ξεκινήσουμε με το επίπεδο **Positional Embedding**. Θα χρησιμοποιήσουμε [κάποιον κώδικα από την επίσημη τεκμηρίωση του Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Θα υποθέσουμε ότι γεμίζουμε όλες τις εισαγόμενες ακολουθίες σε μήκος `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Αυτό το επίπεδο αποτελείται από δύο `Embedding` επίπεδα: για την ενσωμάτωση των tokens (με τρόπο που έχουμε συζητήσει προηγουμένως) και των θέσεων των tokens. Οι θέσεις των tokens δημιουργούνται ως μια ακολουθία φυσικών αριθμών από το 0 έως το `maxlen` χρησιμοποιώντας το `tf.range`, και στη συνέχεια περνούν μέσα από το επίπεδο ενσωμάτωσης. Οι δύο προκύπτοντες διανύσματα ενσωμάτωσης προστίθενται, παράγοντας μια αναπαράσταση με ενσωματωμένες θέσεις για την είσοδο με σχήμα `maxlen`$\times$`embed_dim`.

Τώρα, ας υλοποιήσουμε το μπλοκ του transformer. Θα χρησιμοποιήσει την έξοδο του προηγουμένως ορισμένου επιπέδου ενσωμάτωσης:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Τώρα είμαστε έτοιμοι να ορίσουμε το πλήρες μοντέλο του transformer:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Μοντέλα BERT Transformer

**BERT** (Bidirectional Encoder Representations from Transformers) είναι ένα πολύ μεγάλο πολυεπίπεδο δίκτυο transformer με 12 επίπεδα για το *BERT-base* και 24 για το *BERT-large*. Το μοντέλο εκπαιδεύεται αρχικά σε ένα μεγάλο σύνολο δεδομένων κειμένου (WikiPedia + βιβλία) χρησιμοποιώντας μη επιβλεπόμενη εκπαίδευση (πρόβλεψη λέξεων που έχουν καλυφθεί σε μια πρόταση). Κατά τη διάρκεια της αρχικής εκπαίδευσης, το μοντέλο αποκτά σημαντικό επίπεδο κατανόησης της γλώσσας, το οποίο μπορεί στη συνέχεια να αξιοποιηθεί με άλλα σύνολα δεδομένων μέσω της διαδικασίας της λεπτομερούς προσαρμογής. Αυτή η διαδικασία ονομάζεται **μεταφορά μάθησης**.

![εικόνα από http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/el/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Υπάρχουν πολλές παραλλαγές αρχιτεκτονικών Transformer, όπως BERT, DistilBERT, BigBird, OpenGPT3 και άλλες, που μπορούν να προσαρμοστούν περαιτέρω.

Ας δούμε πώς μπορούμε να χρησιμοποιήσουμε το προεκπαιδευμένο μοντέλο BERT για να λύσουμε το παραδοσιακό μας πρόβλημα ταξινόμησης ακολουθιών. Θα δανειστούμε την ιδέα και κάποιον κώδικα από την [επίσημη τεκμηρίωση](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Για να φορτώσουμε προεκπαιδευμένα μοντέλα, θα χρησιμοποιήσουμε το **Tensorflow hub**. Πρώτα, ας φορτώσουμε τον ειδικό για το BERT vectorizer:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Είναι σημαντικό να χρησιμοποιήσετε τον ίδιο vectorizer που χρησιμοποιήθηκε για την εκπαίδευση του αρχικού δικτύου. Επίσης, ο vectorizer του BERT επιστρέφει τρία στοιχεία:
* `input_word_ids`, που είναι μια ακολουθία αριθμών token για την εισαγόμενη πρόταση
* `input_mask`, που δείχνει ποιο μέρος της ακολουθίας περιέχει πραγματική είσοδο και ποιο είναι padding. Είναι παρόμοιο με τη μάσκα που παράγεται από το επίπεδο `Masking`
* `input_type_ids` χρησιμοποιείται για εργασίες γλωσσικής μοντελοποίησης και επιτρέπει τον καθορισμό δύο προτάσεων εισόδου σε μία ακολουθία.

Στη συνέχεια, μπορούμε να δημιουργήσουμε ένα BERT feature extractor:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Έτσι, το BERT layer επιστρέφει έναν αριθμό χρήσιμων αποτελεσμάτων:
* `pooled_output` είναι το αποτέλεσμα της μέσης όλων των tokens στη σειρά. Μπορείτε να το δείτε ως μια έξυπνη σημασιολογική ενσωμάτωση ολόκληρου του δικτύου. Είναι ισοδύναμο με την έξοδο του layer `GlobalAveragePooling1D` στο προηγούμενο μοντέλο μας.
* `sequence_output` είναι η έξοδος του τελευταίου transformer layer (αντιστοιχεί στην έξοδο του `TransformerBlock` στο μοντέλο μας παραπάνω).
* `encoder_outputs` είναι οι έξοδοι όλων των transformer layers. Εφόσον έχουμε φορτώσει ένα BERT μοντέλο με 4 layers (όπως μπορείτε πιθανώς να μαντέψετε από το όνομα, που περιέχει `4_H`), έχει 4 tensors. Το τελευταίο είναι το ίδιο με το `sequence_output`.

Τώρα θα ορίσουμε το end-to-end μοντέλο ταξινόμησης. Θα χρησιμοποιήσουμε *λειτουργικό ορισμό μοντέλου*, όπου ορίζουμε την είσοδο του μοντέλου και στη συνέχεια παρέχουμε μια σειρά από εκφράσεις για τον υπολογισμό της εξόδου του. Θα κάνουμε επίσης τα βάρη του BERT μοντέλου μη εκπαιδεύσιμα και θα εκπαιδεύσουμε μόνο τον τελικό ταξινομητή:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Παρά το γεγονός ότι υπάρχουν λίγες παραμετροποιήσιμες παράμετροι, η διαδικασία είναι αρκετά αργή, επειδή ο εξαγωγέας χαρακτηριστικών του BERT είναι υπολογιστικά απαιτητικός. Φαίνεται ότι δεν καταφέραμε να επιτύχουμε ικανοποιητική ακρίβεια, είτε λόγω έλλειψης εκπαίδευσης, είτε λόγω έλλειψης παραμέτρων του μοντέλου.

Ας προσπαθήσουμε να ξεπαγώσουμε τα βάρη του BERT και να το εκπαιδεύσουμε επίσης. Αυτό απαιτεί πολύ μικρό ρυθμό εκμάθησης, καθώς και πιο προσεκτική στρατηγική εκπαίδευσης με **warmup**, χρησιμοποιώντας τον βελτιστοποιητή **AdamW**. Θα χρησιμοποιήσουμε το πακέτο `tf-models-official` για να δημιουργήσουμε τον βελτιστοποιητή:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Όπως βλέπετε, η εκπαίδευση προχωρά αρκετά αργά - αλλά ίσως θελήσετε να πειραματιστείτε και να εκπαιδεύσετε το μοντέλο για μερικές εποχές (5-10) και να δείτε αν μπορείτε να πετύχετε το καλύτερο αποτέλεσμα συγκρίνοντας με τις προσεγγίσεις που έχουμε χρησιμοποιήσει προηγουμένως.

## Βιβλιοθήκη Huggingface Transformers

Ένας άλλος πολύ συνηθισμένος (και λίγο πιο απλός) τρόπος χρήσης των μοντέλων Transformer είναι το [πακέτο HuggingFace](https://github.com/huggingface/), το οποίο παρέχει απλά δομικά στοιχεία για διάφορες εργασίες NLP. Είναι διαθέσιμο τόσο για Tensorflow όσο και για PyTorch, ένα άλλο πολύ δημοφιλές πλαίσιο νευρωνικών δικτύων.

> **Σημείωση**: Αν δεν σας ενδιαφέρει να δείτε πώς λειτουργεί η βιβλιοθήκη Transformers - μπορείτε να μεταβείτε στο τέλος αυτού του σημειωματάριου, καθώς δεν θα δείτε κάτι ουσιαστικά διαφορετικό από ό,τι έχουμε κάνει παραπάνω. Θα επαναλάβουμε τα ίδια βήματα εκπαίδευσης του μοντέλου BERT χρησιμοποιώντας διαφορετική βιβλιοθήκη και ένα σημαντικά μεγαλύτερο μοντέλο. Επομένως, η διαδικασία περιλαμβάνει κάποια αρκετά χρονοβόρα εκπαίδευση, οπότε ίσως θελήσετε απλώς να ρίξετε μια ματιά στον κώδικα.

Ας δούμε πώς μπορεί να λυθεί το πρόβλημά μας χρησιμοποιώντας το [Huggingface Transformers](http://huggingface.co).


Το πρώτο πράγμα που πρέπει να κάνουμε είναι να επιλέξουμε το μοντέλο που θα χρησιμοποιήσουμε. Εκτός από κάποια ενσωματωμένα μοντέλα, το Huggingface περιέχει ένα [online αποθετήριο μοντέλων](https://huggingface.co/models), όπου μπορείτε να βρείτε πολλά περισσότερα προεκπαιδευμένα μοντέλα από την κοινότητα. Όλα αυτά τα μοντέλα μπορούν να φορτωθούν και να χρησιμοποιηθούν απλά παρέχοντας το όνομα του μοντέλου. Όλα τα απαραίτητα δυαδικά αρχεία για το μοντέλο θα κατέβουν αυτόματα.

Σε ορισμένες περιπτώσεις θα χρειαστεί να φορτώσετε τα δικά σας μοντέλα, οπότε μπορείτε να καθορίσετε τον κατάλογο που περιέχει όλα τα σχετικά αρχεία, συμπεριλαμβανομένων των παραμέτρων για τον tokenizer, το αρχείο `config.json` με τις παραμέτρους του μοντέλου, τα δυαδικά βάρη, κ.λπ.

Από το όνομα του μοντέλου, μπορούμε να δημιουργήσουμε τόσο το μοντέλο όσο και τον tokenizer. Ας ξεκινήσουμε με τον tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Το αντικείμενο `tokenizer` περιέχει τη συνάρτηση `encode` που μπορεί να χρησιμοποιηθεί άμεσα για την κωδικοποίηση κειμένου:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Μπορούμε επίσης να χρησιμοποιήσουμε τον tokenizer για να κωδικοποιήσουμε μια ακολουθία με τρόπο κατάλληλο για να περάσει στο μοντέλο, δηλαδή συμπεριλαμβάνοντας τα πεδία `token_ids`, `input_mask`, κ.λπ. Μπορούμε επίσης να καθορίσουμε ότι θέλουμε τανυστές Tensorflow παρέχοντας το όρισμα `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Στην περίπτωσή μας, θα χρησιμοποιήσουμε το προεκπαιδευμένο μοντέλο BERT που ονομάζεται `bert-base-uncased`. Το *Uncased* υποδηλώνει ότι το μοντέλο δεν κάνει διάκριση πεζών-κεφαλαίων.

Κατά την εκπαίδευση του μοντέλου, πρέπει να παρέχουμε ως είσοδο ακολουθίες που έχουν υποστεί τοκενισμό, και γι' αυτό θα σχεδιάσουμε μια διαδικασία επεξεργασίας δεδομένων. Επειδή η `tokenizer.encode` είναι μια συνάρτηση της Python, θα χρησιμοποιήσουμε την ίδια προσέγγιση όπως στην προηγούμενη ενότητα, καλώντας την μέσω της `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Τώρα μπορούμε να φορτώσουμε το πραγματικό μοντέλο χρησιμοποιώντας το πακέτο `BertForSequenceClassification`. Αυτό διασφαλίζει ότι το μοντέλο μας έχει ήδη την απαιτούμενη αρχιτεκτονική για ταξινόμηση, συμπεριλαμβανομένου του τελικού ταξινομητή. Θα δείτε ένα μήνυμα προειδοποίησης που δηλώνει ότι τα βάρη του τελικού ταξινομητή δεν έχουν αρχικοποιηθεί και το μοντέλο θα χρειαστεί προεκπαίδευση - αυτό είναι απολύτως εντάξει, γιατί αυτό ακριβώς πρόκειται να κάνουμε!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Όπως μπορείτε να δείτε από το `summary()`, το μοντέλο περιέχει σχεδόν 110 εκατομμύρια παραμέτρους! Πιθανώς, αν θέλουμε μια απλή εργασία ταξινόμησης σε σχετικά μικρό σύνολο δεδομένων, δεν θέλουμε να εκπαιδεύσουμε τη βασική στρώση του BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Τώρα είμαστε έτοιμοι να ξεκινήσουμε την εκπαίδευση!

> **Σημείωση**: Η εκπαίδευση ενός πλήρους μοντέλου BERT μπορεί να είναι πολύ χρονοβόρα! Γι' αυτό θα το εκπαιδεύσουμε μόνο για τις πρώτες 32 παρτίδες. Αυτό γίνεται απλώς για να δείξουμε πώς ρυθμίζεται η εκπαίδευση του μοντέλου. Αν ενδιαφέρεστε να δοκιμάσετε την πλήρη εκπαίδευση - απλώς αφαιρέστε τις παραμέτρους `steps_per_epoch` και `validation_steps`, και ετοιμαστείτε να περιμένετε!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Αν αυξήσετε τον αριθμό των επαναλήψεων και περιμένετε αρκετά, και εκπαιδεύσετε για αρκετές εποχές, μπορείτε να περιμένετε ότι η ταξινόμηση με BERT θα μας δώσει την καλύτερη ακρίβεια! Αυτό συμβαίνει επειδή το BERT ήδη κατανοεί αρκετά καλά τη δομή της γλώσσας, και το μόνο που χρειάζεται είναι να προσαρμόσουμε τον τελικό ταξινομητή. Ωστόσο, επειδή το BERT είναι ένα μεγάλο μοντέλο, όλη η διαδικασία εκπαίδευσης διαρκεί πολύ χρόνο και απαιτεί σημαντική υπολογιστική ισχύ! (GPU, και κατά προτίμηση περισσότερες από μία).

> **Σημείωση:** Στο παράδειγμά μας, χρησιμοποιούμε ένα από τα μικρότερα προεκπαιδευμένα μοντέλα BERT. Υπάρχουν μεγαλύτερα μοντέλα που πιθανότατα θα αποδώσουν καλύτερα αποτελέσματα.


## Συμπέρασμα

Σε αυτήν την ενότητα, εξετάσαμε πολύ πρόσφατες αρχιτεκτονικές μοντέλων βασισμένες στους **transformers**. Τις εφαρμόσαμε για την εργασία ταξινόμησης κειμένου, αλλά αντίστοιχα, τα μοντέλα BERT μπορούν να χρησιμοποιηθούν για εξαγωγή οντοτήτων, απάντηση ερωτήσεων και άλλες εργασίες NLP.

Τα μοντέλα transformers αντιπροσωπεύουν την πιο προηγμένη τεχνολογία στο NLP, και στις περισσότερες περιπτώσεις θα πρέπει να είναι η πρώτη λύση με την οποία ξεκινάτε να πειραματίζεστε όταν υλοποιείτε προσαρμοσμένες λύσεις NLP. Ωστόσο, η κατανόηση των βασικών αρχών των επαναλαμβανόμενων νευρωνικών δικτύων που συζητήθηκαν σε αυτήν την ενότητα είναι εξαιρετικά σημαντική αν θέλετε να δημιουργήσετε προηγμένα νευρωνικά μοντέλα.



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτοματοποιημένες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
